<a href="https://colab.research.google.com/github/Sibusisongwenya/WIP-Project/blob/main/train_cv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys
# Append the project root to the Python path
sys.path.append('/content/drive/MyDrive/uc')
!pip install torchbnn
os.chdir("/content/drive/MyDrive/uc")
print("Current working directory:", os.getcwd())

Current working directory: /content/drive/MyDrive/uc


In [ ]:
import os
import gc
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import KFold

# If you want to use mixed precision (amp), import these:
from torch.cuda.amp import autocast, GradScaler

# Import dataset and model definitions
from dataset.ucmayo4 import UCMayo4
from utils.magic import BayesianDenseNet121_LLSVI, DenseNet121_LLDropout

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ----------- GPU optimization flags -----------
torch.backends.cudnn.benchmark = True  # Let cuDNN optimize for your input sizes
# ----------------------------------------------

# ------------------
# Hyperparameters
# ------------------
EPOCHS = 20
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DROPOUT_PROB = 0.5
K_FOLDS = 10
CHECKPOINT_DIR = "weights_cv/"
USE_AMP = True  # Set to False if you don't want to use mixed precision

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {DEVICE}")

# Data setup
train_dir = "/content/drive/MyDrive/uc/test_set/train"
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
full_dataset = UCMayo4(root_dir=train_dir, transform=transform)
dataset_size = len(full_dataset)
logging.info(f"Full dataset size: {dataset_size} samples")

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Metric definitions
mse_loss = nn.MSELoss()

def compute_mae_rmse(outputs, targets):
    diff = outputs - targets
    mae = diff.abs().mean().item()
    rmse = torch.sqrt((diff ** 2).mean()).item()
    return mae, rmse

def train_epoch(model, dataloader, optimizer, scaler, device):
    model.train()
    running_loss, total_mae, total_rmse = 0.0, 0.0, 0.0
    total_samples = 0

    for images, targets in dataloader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True).float().unsqueeze(1)
        optimizer.zero_grad()

        if USE_AMP:
            # Mixed precision
            with autocast():
                outputs = model(images, sample=False)
                loss = mse_loss(outputs, targets)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
        else:
            outputs = model(images, sample=False)
            loss = mse_loss(outputs, targets)
            loss.backward()
            optimizer.step()

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size
        # track metrics
        with torch.no_grad():
            mae, rmse = compute_mae_rmse(outputs, targets)
            total_mae += mae * batch_size
            total_rmse += rmse * batch_size
        total_samples += batch_size

    epoch_loss = running_loss / total_samples
    epoch_mae  = total_mae / total_samples
    epoch_rmse = total_rmse / total_samples
    return epoch_loss, epoch_mae, epoch_rmse

@torch.no_grad()
def validate_epoch(model, dataloader, device):
    model.eval()
    running_loss, total_mae, total_rmse = 0.0, 0.0, 0.0
    total_samples = 0

    for images, targets in dataloader:
        images = images.to(device, non_blocking=True)
        targets = targets.to(device, non_blocking=True).float().unsqueeze(1)

        outputs = model(images, sample=False)
        loss = mse_loss(outputs, targets)

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size
        mae, rmse = compute_mae_rmse(outputs, targets)
        total_mae += mae * batch_size
        total_rmse += rmse * batch_size
        total_samples += batch_size

    epoch_loss = running_loss / total_samples
    epoch_mae  = total_mae / total_samples
    epoch_rmse = total_rmse / total_samples
    return epoch_loss, epoch_mae, epoch_rmse

kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
indices = np.arange(dataset_size)

fold_results_llsvi = []
fold_results_dropout = []

for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    logging.info(f"\n--- Fold {fold+1}/{K_FOLDS} ---")
    train_subset = Subset(full_dataset, train_idx)
    val_subset   = Subset(full_dataset, val_idx)

    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True,
                              pin_memory=True, num_workers=2)
    val_loader   = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False,
                              pin_memory=True, num_workers=2)

    # Initialize models
    model_llsvi = BayesianDenseNet121_LLSVI(pretrained=True).to(DEVICE)
    model_dropout = DenseNet121_LLDropout(pretrained=True, dropout_prob=DROPOUT_PROB).to(DEVICE)

    optimizer_llsvi = optim.Adam(model_llsvi.parameters(), lr=LEARNING_RATE)
    optimizer_dropout = optim.Adam(model_dropout.parameters(), lr=LEARNING_RATE)

    # For mixed precision
    scaler_llsvi = GradScaler(enabled=USE_AMP)
    scaler_dropout = GradScaler(enabled=USE_AMP)

    best_val_loss_llsvi = float('inf')
    best_val_loss_dropout = float('inf')
    final_val_mae_llsvi, final_val_rmse_llsvi = 0.0, 0.0
    final_val_mae_dropout, final_val_rmse_dropout = 0.0, 0.0

    for epoch in range(EPOCHS):
        logging.info(f"Fold {fold+1}, Epoch {epoch+1}/{EPOCHS}")

        # LLSVI train/val
        train_loss_llsvi, train_mae_llsvi, train_rmse_llsvi = train_epoch(
            model_llsvi, train_loader, optimizer_llsvi, scaler_llsvi, DEVICE
        )
        val_loss_llsvi, val_mae_llsvi, val_rmse_llsvi = validate_epoch(
            model_llsvi, val_loader, DEVICE
        )

        # Dropout train/val
        train_loss_dropout, train_mae_dropout, train_rmse_dropout = train_epoch(
            model_dropout, train_loader, optimizer_dropout, scaler_dropout, DEVICE
        )
        val_loss_dropout, val_mae_dropout, val_rmse_dropout = validate_epoch(
            model_dropout, val_loader, DEVICE
        )

        logging.info(
          f"LL-SVI    [E{epoch+1}] Train MSE: {train_loss_llsvi:.4f},"
          f" Val MSE: {val_loss_llsvi:.4f}, MAE: {val_mae_llsvi:.4f}, RMSE: {val_rmse_llsvi:.4f}"
        )
        logging.info(
          f"Dropout   [E{epoch+1}] Train MSE: {train_loss_dropout:.4f},"
          f" Val MSE: {val_loss_dropout:.4f}, MAE: {val_mae_dropout:.4f}, RMSE: {val_rmse_dropout:.4f}"
        )

        # Save best fold checkpoint for LLSVI
        if val_loss_llsvi < best_val_loss_llsvi:
            best_val_loss_llsvi = val_loss_llsvi
            final_val_mae_llsvi, final_val_rmse_llsvi = val_mae_llsvi, val_rmse_llsvi
            ckpt_path_llsvi = os.path.join(CHECKPOINT_DIR, f"fold_{fold+1}_best_llsvi.pth")
            torch.save(model_llsvi.state_dict(), ckpt_path_llsvi)

        # Save best fold checkpoint for Dropout
        if val_loss_dropout < best_val_loss_dropout:
            best_val_loss_dropout = val_loss_dropout
            final_val_mae_dropout, final_val_rmse_dropout = val_mae_dropout, val_rmse_dropout
            ckpt_path_dropout = os.path.join(CHECKPOINT_DIR, f"fold_{fold+1}_best_dropout.pth")
            torch.save(model_dropout.state_dict(), ckpt_path_dropout)

    # Accumulate final results from each fold
    fold_results_llsvi.append((best_val_loss_llsvi, final_val_mae_llsvi, final_val_rmse_llsvi))
    fold_results_dropout.append((best_val_loss_dropout, final_val_mae_dropout, final_val_rmse_dropout))

    # Free up memory, especially if you have big models
    del model_llsvi, model_dropout
    torch.cuda.empty_cache()
    gc.collect()

# Summarize cross-fold results
fold_results_llsvi = np.array(fold_results_llsvi)  # shape: (K_FOLDS, 3) => [mse, mae, rmse]
fold_results_dropout = np.array(fold_results_dropout)

mean_llsvi = np.mean(fold_results_llsvi, axis=0)
std_llsvi  = np.std(fold_results_llsvi, axis=0)
mean_dropout = np.mean(fold_results_dropout, axis=0)
std_dropout  = np.std(fold_results_dropout, axis=0)

logging.info("\n=== Cross-Validation Results (LL-SVI) ===")
logging.info(f"MSE : {mean_llsvi[0]:.4f} +/- {std_llsvi[0]:.4f}")
logging.info(f"MAE : {mean_llsvi[1]:.4f} +/- {std_llsvi[1]:.4f}")
logging.info(f"RMSE: {mean_llsvi[2]:.4f} +/- {std_llsvi[2]:.4f}")

logging.info("\n=== Cross-Validation Results (MC-Dropout) ===")
logging.info(f"MSE : {mean_dropout[0]:.4f} +/- {std_dropout[0]:.4f}")
logging.info(f"MAE : {mean_dropout[1]:.4f} +/- {std_dropout[1]:.4f}")
logging.info(f"RMSE: {mean_dropout[2]:.4f} +/- {std_dropout[2]:.4f}")

logging.info("Cross-validation complete.")


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 77.4MB/s]
<ipython-input-3-6c1a41e2d73a>:149: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler_llsvi =

In [ ]:
import os
import gc
import logging
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import transforms
from sklearn.model_selection import KFold

# WandB import
import wandb

from dataset.ucmayo4 import UCMayo4
from utils.magic import BayesianDenseNet121_LLSVI, DenseNet121_LLDropout

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ------------------
# Hyperparameters
# ------------------
EPOCHS = 5
BATCH_SIZE = 32
LEARNING_RATE = 1e-3
DROPOUT_PROB = 0.5
K_FOLDS = 10
CHECKPOINT_DIR = "weights_cv/"
PROJECT_NAME = "my-ucmayo4-cv"  # Name for your wandb project

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.info(f"Using device: {DEVICE}")

# Data setup
train_dir = "/content/drive/MyDrive/uc/test_set/train"
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
full_dataset = UCMayo4(root_dir=train_dir, transform=transform)
dataset_size = len(full_dataset)
logging.info(f"Full dataset size: {dataset_size} samples")

os.makedirs(CHECKPOINT_DIR, exist_ok=True)

# Define MSE plus additional metrics
mse_loss = nn.MSELoss()

def compute_mae_rmse(outputs, targets):
    diff = outputs - targets
    mae = diff.abs().mean().item()
    rmse = torch.sqrt((diff**2).mean()).item()
    return mae, rmse

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss, total_samples = 0.0, 0
    total_mae, total_rmse = 0.0, 0.0

    for images, targets in dataloader:
        images = images.to(device)
        targets = targets.to(device).float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images, sample=False)  # normal forward
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size
        mae_batch, rmse_batch = compute_mae_rmse(outputs, targets)
        total_mae += mae_batch * batch_size
        total_rmse += rmse_batch * batch_size
        total_samples += batch_size

    epoch_loss = running_loss / total_samples
    epoch_mae  = total_mae / total_samples
    epoch_rmse = total_rmse / total_samples
    return epoch_loss, epoch_mae, epoch_rmse

@torch.no_grad()
def validate_epoch(model, dataloader, criterion, device):
    model.eval()
    running_loss, total_samples = 0.0, 0
    total_mae, total_rmse = 0.0, 0.0

    for images, targets in dataloader:
        images = images.to(device)
        targets = targets.to(device).float().unsqueeze(1)
        outputs = model(images, sample=False)
        loss = criterion(outputs, targets)

        batch_size = images.size(0)
        running_loss += loss.item() * batch_size
        mae_batch, rmse_batch = compute_mae_rmse(outputs, targets)
        total_mae += mae_batch * batch_size
        total_rmse += rmse_batch * batch_size
        total_samples += batch_size

    epoch_loss = running_loss / total_samples
    epoch_mae  = total_mae / total_samples
    epoch_rmse = total_rmse / total_samples
    return epoch_loss, epoch_mae, epoch_rmse


kf = KFold(n_splits=K_FOLDS, shuffle=True, random_state=42)
indices = np.arange(dataset_size)

fold_results_llsvi = []
fold_results_dropout = []

for fold, (train_idx, val_idx) in enumerate(kf.split(indices)):
    logging.info(f"\n--- Fold {fold+1}/{K_FOLDS} ---")

    # Initialize a new wandb run for each fold
    wandb_run = wandb.init(
        project=PROJECT_NAME,
        name=f"fold_{fold+1}",
        config={
            "epochs": EPOCHS,
            "batch_size": BATCH_SIZE,
            "learning_rate": LEARNING_RATE,
            "dropout_prob": DROPOUT_PROB,
            "fold_index": fold+1
        },
        reinit=True
    )

    train_subset = Subset(full_dataset, train_idx)
    val_subset   = Subset(full_dataset, val_idx)
    train_loader = DataLoader(train_subset, batch_size=BATCH_SIZE, shuffle=True)
    val_loader   = DataLoader(val_subset, batch_size=BATCH_SIZE, shuffle=False)

    # Initialize models
    model_llsvi = BayesianDenseNet121_LLSVI(pretrained=True).to(DEVICE)
    model_dropout = DenseNet121_LLDropout(pretrained=True, dropout_prob=DROPOUT_PROB).to(DEVICE)

    optimizer_llsvi = optim.Adam(model_llsvi.parameters(), lr=LEARNING_RATE)
    optimizer_dropout = optim.Adam(model_dropout.parameters(), lr=LEARNING_RATE)

    best_val_loss_llsvi = float('inf')
    best_val_loss_dropout = float('inf')
    final_val_mae_llsvi, final_val_rmse_llsvi = 0.0, 0.0
    final_val_mae_dropout, final_val_rmse_dropout = 0.0, 0.0

    for epoch in range(EPOCHS):
        logging.info(f"Fold {fold+1}, Epoch {epoch+1}/{EPOCHS}")

        # Train LLSVI
        train_loss_llsvi, train_mae_llsvi, train_rmse_llsvi = train_epoch(
            model_llsvi, train_loader, mse_loss, optimizer_llsvi, DEVICE
        )
        # Validate LLSVI
        val_loss_llsvi, val_mae_llsvi, val_rmse_llsvi = validate_epoch(
            model_llsvi, val_loader, mse_loss, DEVICE
        )

        # Train MC-Dropout
        train_loss_dropout, train_mae_dropout, train_rmse_dropout = train_epoch(
            model_dropout, train_loader, mse_loss, optimizer_dropout, DEVICE
        )
        # Validate MC-Dropout
        val_loss_dropout, val_mae_dropout, val_rmse_dropout = validate_epoch(
            model_dropout, val_loader, mse_loss, DEVICE
        )

        # Log metrics to wandb
        wandb.log({
            f"fold_{fold+1}/epoch": epoch+1,

            # LLSVI
            f"fold_{fold+1}/train_loss_llsvi": train_loss_llsvi,
            f"fold_{fold+1}/val_loss_llsvi": val_loss_llsvi,
            f"fold_{fold+1}/train_mae_llsvi": train_mae_llsvi,
            f"fold_{fold+1}/val_mae_llsvi": val_mae_llsvi,
            f"fold_{fold+1}/train_rmse_llsvi": train_rmse_llsvi,
            f"fold_{fold+1}/val_rmse_llsvi": val_rmse_llsvi,

            # Dropout
            f"fold_{fold+1}/train_loss_dropout": train_loss_dropout,
            f"fold_{fold+1}/val_loss_dropout": val_loss_dropout,
            f"fold_{fold+1}/train_mae_dropout": train_mae_dropout,
            f"fold_{fold+1}/val_mae_dropout": val_mae_dropout,
            f"fold_{fold+1}/train_rmse_dropout": train_rmse_dropout,
            f"fold_{fold+1}/val_rmse_dropout": val_rmse_dropout
        }, step=epoch+1)

        logging.info(
          f"[LL-SVI: fold {fold+1}, epoch {epoch+1}] "
          f"Train MSE: {train_loss_llsvi:.4f}, Val MSE: {val_loss_llsvi:.4f}, "
          f"MAE: {val_mae_llsvi:.4f}, RMSE: {val_rmse_llsvi:.4f}"
        )
        logging.info(
          f"[Dropout: fold {fold+1}, epoch {epoch+1}] "
          f"Train MSE: {train_loss_dropout:.4f}, Val MSE: {val_loss_dropout:.4f}, "
          f"MAE: {val_mae_dropout:.4f}, RMSE: {val_rmse_dropout:.4f}"
        )

        # Save best fold checkpoint for LLSVI
        if val_loss_llsvi < best_val_loss_llsvi:
            best_val_loss_llsvi = val_loss_llsvi
            final_val_mae_llsvi, final_val_rmse_llsvi = val_mae_llsvi, val_rmse_llsvi
            torch.save(model_llsvi.state_dict(), os.path.join(CHECKPOINT_DIR, f"fold_{fold+1}_best_llsvi.pth"))

        # Save best fold checkpoint for Dropout
        if val_loss_dropout < best_val_loss_dropout:
            best_val_loss_dropout = val_loss_dropout
            final_val_mae_dropout, final_val_rmse_dropout = val_mae_dropout, val_rmse_dropout
            torch.save(model_dropout.state_dict(), os.path.join(CHECKPOINT_DIR, f"fold_{fold+1}_best_dropout.pth"))

    # Store final results for each model in this fold
    fold_results_llsvi.append((best_val_loss_llsvi, final_val_mae_llsvi, final_val_rmse_llsvi))
    fold_results_dropout.append((best_val_loss_dropout, final_val_mae_dropout, final_val_rmse_dropout))

    # End wandb run for this fold
    wandb_run.finish()

    # Clean up memory
    del model_llsvi, model_dropout
    torch.cuda.empty_cache()
    gc.collect()

# Summarize cross-fold results
fold_results_llsvi = np.array(fold_results_llsvi)  # shape (K_FOLDS, 3) => columns = [val_mse, val_mae, val_rmse]
fold_results_dropout = np.array(fold_results_dropout)

mean_llsvi = np.mean(fold_results_llsvi, axis=0)
std_llsvi  = np.std(fold_results_llsvi, axis=0)
mean_dropout = np.mean(fold_results_dropout, axis=0)
std_dropout  = np.std(fold_results_dropout, axis=0)

logging.info("\n=== Cross-Validation Results (LL-SVI) ===")
logging.info(f"MSE : {mean_llsvi[0]:.4f} +/- {std_llsvi[0]:.4f}")
logging.info(f"MAE : {mean_llsvi[1]:.4f} +/- {std_llsvi[1]:.4f}")
logging.info(f"RMSE: {mean_llsvi[2]:.4f} +/- {std_llsvi[2]:.4f}")

logging.info("\n=== Cross-Validation Results (MC-Dropout) ===")
logging.info(f"MSE : {mean_dropout[0]:.4f} +/- {std_dropout[0]:.4f}")
logging.info(f"MAE : {mean_dropout[1]:.4f} +/- {std_dropout[1]:.4f}")
logging.info(f"RMSE: {mean_dropout[2]:.4f} +/- {std_dropout[2]:.4f}")

logging.info("Cross-validation complete.")


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: fngwenya (fngwenya-z) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DenseNet121_Weights.IMAGENET1K_V1`. You can also use `weights=DenseNet121_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 153MB/s]


fold_1/epoch,▁▃▅▆█
fold_1/train_loss_dropout,█▃▂▂▁
fold_1/train_loss_llsvi,█▅▂▃▁
fold_1/train_mae_dropout,█▃▂▂▁
fold_1/train_mae_llsvi,█▄▂▂▁
fold_1/train_rmse_dropout,█▃▂▂▁
fold_1/train_rmse_llsvi,█▅▃▂▁
fold_1/val_loss_dropout,▂█▁▃▅
fold_1/val_loss_llsvi,▁▁▁▁█
fold_1/val_mae_dropout,▄█▁▃▆
fold_1/val_mae_llsvi,▁▁▁▁█


fold_2/epoch,▁▃▅▆█
fold_2/train_loss_dropout,█▄▂▁▁
fold_2/train_loss_llsvi,█▃▃▁▃
fold_2/train_mae_dropout,█▄▂▁▁
fold_2/train_mae_llsvi,█▂▃▁▃
fold_2/train_rmse_dropout,█▄▂▁▁
fold_2/train_rmse_llsvi,█▃▃▁▃
fold_2/val_loss_dropout,▃▂▁▃█
fold_2/val_loss_llsvi,▁▂▁▁█
fold_2/val_mae_dropout,▃▂▁▄█
fold_2/val_mae_llsvi,▂▇▂▁█


fold_3/epoch,▁▃▅▆█
fold_3/train_loss_dropout,█▃▂▁▁
fold_3/train_loss_llsvi,█▅▃▃▁
fold_3/train_mae_dropout,█▄▂▁▁
fold_3/train_mae_llsvi,█▅▃▄▁
fold_3/train_rmse_dropout,█▄▂▁▁
fold_3/train_rmse_llsvi,█▅▃▄▁
fold_3/val_loss_dropout,▂▁▁█▂
fold_3/val_loss_llsvi,▂▁▁█▁
fold_3/val_mae_dropout,▅▂▁█▃
fold_3/val_mae_llsvi,▂▂▁█▁
